In [1]:
import requests
from collections import defaultdict
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='my_app')
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from matplotlib import cm
import networkx as nx
import osmnx as ox
import geopandas as gpd
import networkx as nx
from shapely.geometry.polygon import Point, Polygon
from shapely.geometry import shape
from shapely.ops import unary_union
from shapely.geometry.multipolygon import MultiPolygon
import folium
import geopandas as gpd
import math
from geopy.distance import distance

/Users/paulcarroll/Github/nhs_time_of_travel/geospat/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
lsoa_data = pd.read_csv('/Users/paulcarroll/Github/pycom/nhs_time_of_travel/data/lsoa_global_number_residents_2021.csv')

In [4]:
lsoa_postcode = pd.read_csv('/Users/paulcarroll/Github/pycom/nhs_time_of_travel/data/pcd_lsoa21cd_nov22_en.csv')
lsoa_postcode.head(2)

,POSTCODE,LSOA21CD
0,AL1 1AA,E01023743
1,AL1 1AB,E01023667


In [5]:
lsoa_pop = pd.read_csv('/Users/paulcarroll/Github/pycom/nhs_time_of_travel/data/lsoa_global_number_residents_2021.csv')
lsoa_pop.head(3)

,2021 super output area - lower layer,LSOA21CD,Population
0,Hartlepool 001A,E01011954,2284.0
1,Hartlepool 001B,E01011969,1344.0
2,Hartlepool 001C,E01011970,1070.0


In [6]:

G = ox.graph_from_place('Cambridgeshire', network_type='drive')


/Users/paulcarroll/Github/nhs_time_of_travel/geospat/lib/python3.10/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [30]:


gdf = gpd.read_file('/Users/paulcarroll/Github/pycom/nhs_time_of_travel/data/LSOA_Dec_2021_Boundaries_Generalised_Clipped_EW_BGC_2022_5000101660793162025/LSOA_2021_EW_BGC.shp')
gdf_cambs = gdf.query("LSOA21NM.str.contains('Cambridge')")


In [28]:
gdf_c= gdf.query("LSOA21NM.str.contains('Cambridge')")

In [29]:
len(gdf_c)

228

In [31]:
# set the CRS of the GeoDataFrame to British National Grid (EPSG:27700)
gdf_cambs = gdf_cambs.set_crs(epsg=27700)

# project the geometry to WGS84 (EPSG:4326)
gdf_cambs = gdf_cambs.to_crs(epsg=4326)


lsoa_codes = gdf_cambs['LSOA21CD'].tolist()
lsoa_data = {lsoa_code: {} for lsoa_code in lsoa_codes}


In [32]:
gdf_cambs

,LSOA21CD,LSOA21NM,GlobalID,geometry
17031,E01017943,Cambridge 006A,9adca991-f45d-47e5-8d9b-be0fc67e7c91,"POLYGON ((0.14350 52.21221, 0.14684 52.21113, ..."
17032,E01017944,Cambridge 006B,cf4eb266-0de5-4efe-bbcc-dc202aeb48fc,"POLYGON ((0.16308 52.21952, 0.16380 52.21909, ..."
17033,E01017945,Cambridge 006C,3b7bdbe7-73a6-4e7e-9154-a2a34261bb3a,"POLYGON ((0.15525 52.21761, 0.15557 52.21700, ..."
17034,E01017946,Cambridge 006D,6f775e02-7dc9-4ef7-933f-6adaa9c83fba,"POLYGON ((0.16122 52.21695, 0.16250 52.21562, ..."
17035,E01017947,Cambridge 006E,8eab504d-aa88-4b10-8b58-1ae0f854f2d3,"POLYGON ((0.16743 52.21316, 0.16707 52.21246, ..."
...,...,...,...,...
33550,E01035557,South Cambridgeshire 003G,979fcdb2-e234-4e62-bc9c-aad3631ed525,"POLYGON ((0.04833 52.28249, 0.04954 52.28094, ..."
33551,E01035558,South Cambridgeshire 003H,3a558ac7-33c9-4d2c-ba8d-28413977e380,"POLYGON ((0.03691 52.28442, 0.03989 52.28176, ..."
33552,E01035559,South Cambridgeshire 020F,347ab23b-4e21-43bb-b4a5-f0482cbc8a4e,"POLYGON ((-0.05315 52.22067, -0.05397 52.21965..."
33553,E01035560,South Cambridgeshire 020G,ab2b9025-ffbd-41a3-9bb8-93e8424e4ac4,"POLYGON ((-0.05474 52.22286, -0.03962 52.22043..."


In [33]:

for index, row in gdf_cambs.iterrows():
    lsoa_code = row['LSOA21CD']
    if lsoa_code in lsoa_data:
        lsoa_data[lsoa_code]['Latitude'] = row['geometry'].centroid.y
        lsoa_data[lsoa_code]['Longitude'] = row['geometry'].centroid.x
        node = ox.distance.nearest_nodes(G, row['geometry'].centroid.x, row['geometry'].centroid.y)
        lsoa_data[lsoa_code]['Node'] = node


In [34]:
lsoa_population = {}
for lsoa_code in lsoa_data:
    population = lsoa_pop.loc[lsoa_pop['LSOA21CD'] == lsoa_code, 'Population'].iloc[0]
    try:
        latitude = lsoa_data[lsoa_code]['Latitude']
    except KeyError:
        print(f'KeyError: Latitude not found for LSOA code {lsoa_code}')
        latitude = None
    try:
        longitude = lsoa_data[lsoa_code]['Longitude']
    except KeyError:
        print(f'KeyError: Longitude not found for LSOA code {lsoa_code}')
        longitude = None
    lsoa_population[lsoa_code] = {'Population': population, 'Latitude': latitude, 'Longitude': longitude}




In [35]:
lsoa_population

{'E01017943': {'Population': 2182.0,
  'Latitude': 52.210055225300096,
  'Longitude': 0.14383716374683533},
 'E01017944': {'Population': 1432.0,
  'Latitude': 52.21725751784076,
  'Longitude': 0.16246058848020145},
 'E01017945': {'Population': 1742.0,
  'Latitude': 52.21261810586651,
  'Longitude': 0.15283168372231054},
 'E01017946': {'Population': 1574.0,
  'Latitude': 52.21296639170579,
  'Longitude': 0.1598446571944757},
 'E01017947': {'Population': 1730.0,
  'Latitude': 52.20409014099948,
  'Longitude': 0.1675610508210118},
 'E01017948': {'Population': 1682.0,
  'Latitude': 52.21463868805174,
  'Longitude': 0.17084712271090863},
 'E01017949': {'Population': 1607.0,
  'Latitude': 52.21772836960141,
  'Longitude': 0.11320101319679464},
 'E01017950': {'Population': 1297.0,
  'Latitude': 52.21363449304136,
  'Longitude': 0.11642865805626877},
 'E01017951': {'Population': 1500.0,
  'Latitude': 52.227329172723444,
  'Longitude': 0.11122407920361684},
 'E01017952': {'Population': 1652.0,


In [36]:
address= '184 Victoria Road, Cambridge, UK'
radius = 5000
lsoa_data=lsoa_data
G=G
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
#get_average_travel_times('184 Victoria Road, Cambridge, UK',5000,lsoa_data,G)


In [37]:
from geopy.geocoders import Nominatim
import networkx as nx
import osmnx as ox
from haversine import haversine, Unit

geolocator = Nominatim(user_agent='my_application')
location = Nominatim(user_agent='my_app').geocode(address)
origin = (location.latitude, location.longitude)

def haversine_distance(origin, destination, speed=40):


    distance_km = haversine(origin, destination, unit='km')
    time_sec = (distance_km / speed) * 3600
    return time_sec

# get LSOAs within search radius
lsoas_in_radius = []
for lsoa_code, data in lsoa_data.items():
    if haversine_distance(origin, (data['Latitude'], data['Longitude'])) <= radius:
        lsoas_in_radius.append(lsoa_code)

# create a list of LSOA codes within search radius
lsoa_codes_in_radius = [lsoa for lsoa in lsoa_population.keys() if lsoa in lsoas_in_radius]





In [50]:
from geopy.distance import great_circle

def get_average_travel_times(address, radius, lsoa_population, G):


    geolocator = Nominatim(user_agent='my_application')
    location = Nominatim(user_agent='my_app').geocode(address)
    origin = (location.latitude, location.longitude)


    lsoas_in_radius = []

    for lsoa_code, data in lsoa_population.items():
        lsoa_coordinates = (data['Latitude'], data['Longitude'])
        if great_circle(origin, lsoa_coordinates).km <= radius / 1000:
            lsoas_in_radius.append(lsoa_code)


    # calculate shortest paths from origin to LSOAs within search radius
    travel_times = []
    for lsoa_code in lsoas_in_radius:
        destination = (lsoa_population[lsoa_code]['Latitude'], lsoa_population[lsoa_code]['Longitude'])
        try:
            route = nx.shortest_path(G, source=ox.distance.nearest_nodes(G, origin[1], origin[0]),
                                     target=ox.distance.nearest_nodes(G, destination[1], destination[0]), 
                                     weight='travel_time')
            travel_time = sum([G[u][v][0]['travel_time'] for u, v in zip(route[:-1], route[1:])])
            travel_times.append(travel_time)
        except nx.NetworkXNoPath:
            pass

    # calculate average travel time and population within search radius
    avg_travel_time = sum(travel_times) / len(travel_times) / 60
    population_covered = sum([lsoa_population[lsoa]['Population'] for lsoa in lsoas_in_radius])

    return avg_travel_time, population_covered


In [51]:
avg_travel_time, pop_covered = get_average_travel_times('184 Victoria Road, Cambridge, UK', 5000, lsoa_population, G)
print(f"Average travel time: {avg_travel_time} minutes")
print(f"Population covered: {pop_covered}")


Average travel time: 4.856379310344827 minutes
Population covered: 160724.0


In [39]:
total_population = sum([data['Population'] for lsoa, data in lsoa_population.items()])
total_population

395545.0

In [52]:
radius

5000

In [53]:
len(lsoas_in_radius)

87

In [55]:
import folium

location = ox.geocode('184 Victoria Road, Cambridge, UK')

# Create the map
m = folium.Map(location=location, zoom_start=12)
folium.Marker(location=location, tooltip='Address').add_to(m)
folium.Circle(location=location, radius=10000, color='red', fill=False, tooltip='Search Radius').add_to(m)

for lsoa_code in lsoas_in_radius:
    row = gdf_cambs.loc[gdf_cambs['LSOA21CD'] == lsoa_code].iloc[0]
    
    if row['geometry'].geom_type == 'Polygon':
        lsoa_boundary = [tuple(reversed(coord)) for coord in list(row['geometry'].exterior.coords)]
    elif row['geometry'].geom_type == 'MultiPolygon':
        largest_polygon = max(row['geometry'], key=lambda x: x.area) 
        lsoa_boundary = [tuple(reversed(coord)) for coord in list(largest_polygon.exterior.coords)]


    lsoa_polygon = folium.Polygon(
        locations=lsoa_boundary,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.2,
        tooltip=lsoa_code,
    )

   
    lsoa_polygon.add_to(m)


m

/var/folders/r9/ds0v4s794fx8njpdn0nbq9dc0000gn/T/ipykernel_86407/685315124.py:22: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  largest_polygon = max(row['geometry'], key=lambda x: x.area) # Get the largest polygon in the MultiPolygon


In [54]:
print(f"Average travel time: {avg_travel_time} minutes")
print(f"Population covered: {pop_covered}")

Average travel time: 4.856379310344827 minutes
Population covered: 160724.0


In [42]:
import folium
location = ox.geocode('184 Victoria Road, Cambridge, UK')
m = folium.Map(location=location, zoom_start=12)
folium.Marker(location=location, tooltip='Address').add_to(m)
folium.Circle(location=location, radius=10000, color='red', fill=False, tooltip='Search Radius').add_to(m)


In [43]:

for lsoa_code in lsoas_in_radius:
    lsoa_data = lsoa_population[lsoa_code]
    popup_text = f"LSOA code: {lsoa_code}<br>Population: {lsoa_data['Population']}"
    folium.Marker(location=[lsoa_data['Latitude'], lsoa_data['Longitude']], 
                  popup=popup_text, 
                  tooltip=lsoa_data['Population']).add_to(m)

# add LSOA boundaries to map
folium.GeoJson(gdf_cambs[gdf_cambs['LSOA21CD'].isin(lsoas_in_radius)]).add_to(m)


/var/folders/r9/ds0v4s794fx8njpdn0nbq9dc0000gn/T/ipykernel_86407/487638.py:22: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  largest_polygon = max(row['geometry'], key=lambda x: x.area) # Get the largest polygon in the MultiPolygon
/var/folders/r9/ds0v4s794fx8njpdn0nbq9dc0000gn/T/ipykernel_86407/487638.py:22: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  largest_polygon = max(row['geometry'], key=lambda x: x.area) # Get the largest polygon in the MultiPolygon
/var/folders/r9/ds0v4s794fx8njpdn0nbq9dc0000gn/T/ipykernel_86407/487638.py:22: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access th

In [44]:
m

In [ ]:
  #geocode address using Nominatim
geolocator = Nominatim(user_agent='my_application')
location = Nominatim(user_agent='my_app').geocode(address)
origin = (location.latitude, location.longitude)
# get LSOAs within search radius


In [ ]:
lsoas_in_radius = []
for lsoa_code, data in lsoa_population.items():
    if distance(origin, (data['Latitude'], data['Longitude'])).km <= radius/1000:
        lsoas_in_radius.append(lsoa_code)
# calculate shortest paths from origin to LSOAs within search radius


In [ ]:
len(lsoa_population)

In [ ]:
distance(origin, (data['Latitude'], data['Longitude']))

In [ ]:
lsoas_in_radius

In [ ]:
travel_times = []
for lsoa_code in lsoas_in_radius:
    destination = (lsoa_population[lsoa_code]['Latitude'], lsoa_population[lsoa_code]['Longitude'])
    try:
        route = nx.shortest_path(G, source=ox.distance.nearest_nodes(G, origin[1], origin[0]),
                                 target=ox.distance.nearest_nodes(G, destination[1], destination[0]), 
                                 weight='travel_time')
        travel_time = sum([G[u][v][0]['travel_time'] for u, v in zip(route[:-1], route[1:])])
        travel_times.append(travel_time)
    except nx.NetworkXNoPath:
        pass


In [ ]:
# calculate average travel time and population within search radius
avg_travel_time = sum(travel_times) / len(travel_times) / 60
population_covered = sum([lsoa_population[lsoa]['Population'] for lsoa in lsoas_in_radius])

In [ ]:
lsoas_and_populations = {lsoa: lsoa_population[lsoa]['Population'] for lsoa in lsoas_in_radius}


In [ ]:
lsoas_and_populations

In [ ]:
population_covered = sum([lsoa_population[lsoa]['Population'] for lsoa in lsoas_in_radius])

